In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install peft
!pip install bitsandbytes
!pip install sentencePiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.7 MB/s eta 0:00:00


In [ ]:
### 역자 주: 모델 실행시 결과창에서 custom code를 실행할 것이냐고 물어오면 소문자 y를 입력하고 엔터키를 누릅니다.
import torch
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

### 역자 주: 아래 주석 표기한 원서의 Llama-7b-hf 모델은 접근 권한을 얻으려면 신청 후 상당한 시일을 기다려야 합니다.
### 따라서 여기서는 gpt2 모델로 대체합니다.
#model_name = "decapoda-research/llama-7b-hf"
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token

# 데이터셋 불러오기
dataset = load_dataset("tiny_shakespeare")
'''
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 1
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})
'''

# 연속형 텍스트를 작은 청크로 분리
def split_text(text, max_length=100):
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

# split_text 함수를 데이터셋에 적용
split_texts = split_text(dataset["train"]["text"][0])

# split_texts 변수에 담긴 값을 토큰화
tokenized_texts = tokenizer(split_texts, return_tensors="pt",
                            padding=True, truncation=True)

class ShiftedDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        input_ids = self.encodings["input_ids"][idx]
        attention_mask = self.encodings["attention_mask"][idx]
        labels = input_ids[1:].tolist() + [tokenizer.eos_token_id]
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": torch.tensor(labels)}

    def __len__(self):
        return len(self.encodings["input_ids"])

# DataLoader 생성
train_dataset = ShiftedDataset(tokenized_texts)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

tiny_shakespeare.py:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

The repository for tiny_shakespeare contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tiny_shakespeare.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
item=next(iter(train_dataloader))
print(item['input_ids'])
print(item['attention_mask'])
print(item['labels'])

tensor([[  339,   373,   407,  4642,   284, 10195,    25,   198, 23792,   465,
          4772, 10195,   318, 22461,   284,  1650,    26,   198,  1890,   705,
         48010,   257, 19262,   810, 15393,   743, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256],
        [   42,    25,   198,  5195,    11,   703,   783,    11, 11989,   290,
          3956,     0,   379,   257, 39809,    30,   198,  2061,   318,   534,
         41998,    30,   703,  2540,   340,   717,    30,   198,   198,  1961,
         39743,    25,   198,    45, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256],
        [  257, 20785,  1296,    11,   198,   818,  4167,    11,   284,   465,
         27878, 25830,    13,   198,   198,  5962,  8962,    25,   19

In [ ]:
from accelerate import Accelerator
from transformers import GPT2LMHeadModel

# Accelerator 초기화
accelerator = Accelerator()

# training arguments 설정
num_epochs = 20
learning_rate = 5e-5

# GPT-2 모델 및 옵티마이저 초기화
model = GPT2LMHeadModel.from_pretrained("gpt2")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Accelerator와 함께 학습시키기 위해 모델과 옵티마이저 준비
model, optimizer, train_dataloader = accelerator.prepare(model,
                                                         optimizer,
                                                         train_dataloader)


In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### 원서에서는 num_epoch=40, epoch=20으로 설정되어 있으나
### 여기서는 런타임 단축을 위해 둘 다 10으로 설정
num_epochs=10
epoch=10

In [ ]:
# 주의! 런타임 45분 소요

from transformers import AdamW
from tqdm import tqdm

# 파인튜닝 반복 루프
for epoch in range(num_epochs):
    epoch_iterator = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")
    for step, batch in enumerate(epoch_iterator):
        optimizer.zero_grad()
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        accelerator.backward(loss)
        optimizer.step()

        if step % 500 == 0:
            epoch_iterator.set_postfix({"Loss": loss.item()}, refresh=True)

    # 매 5회 에포크마다 모델 저장
    # 여러분 구글 드라이브의 알맞은 경로로 model_save_path 지정
    if (epoch + 1) % 5 == 0:
        model_save_path = f"/content/drive/MyDrive/Book6/Ch6/model_checkpoint_epoch_{epoch + 1}"
        model.save_pretrained(model_save_path)
        print(f"Model saved at epoch {epoch + 1}")


Epoch 5: 100%|██████████| 2510/2510 [04:14<00:00,  9.86it/s, Loss=0.904]


Model saved at epoch 5


Epoch 10: 100%|██████████| 2510/2510 [04:14<00:00,  9.86it/s, Loss=0.283]


Model saved at epoch 10


In [ ]:
### 원서에서는 num_epoch=40, epoch=20으로 설정되어 있으나
### 여기서는 런타임 단축을 위해 둘 다 10으로 설정

num_epochs=10
epoch=10

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)

### 역자 주: 다음 두 줄 코드는 역자가 삽입
model_path = '/content/drive/MyDrive/Book6/Ch6/model_checkpoint_epoch_10'
tokenizer_path = 'gpt2'

unwrapped_model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)


('gpt2/tokenizer_config.json',
 'gpt2/special_tokens_map.json',
 'gpt2/vocab.json',
 'gpt2/merges.txt',
 'gpt2/added_tokens.json',
 'gpt2/tokenizer.json')

In [ ]:
import torch

def generate_poem(prompt, model_path, tokenizer_path, max_words=50, max_seq_len=100, temperature=1.0):
    # 파인튜닝 모델 및 토크나이저 불러오기
    model = GPT2LMHeadModel.from_pretrained(model_path)
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # padding token 및 padding side 설정
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'

    poem = ""
    remaining_words = max_words

    while remaining_words > 0:
        # 프롬프트 설정 및 text 생성
        input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True, max_length=max_seq_len)
        attention_mask = torch.ones_like(input_ids)

        max_tokens = min(remaining_words * 5, max_seq_len)  # 각 단어가 평균 5개 토큰으로 구성됐다고 가정
        output_ids = model.generate(
            input_ids,
            max_length=max_tokens,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            attention_mask=attention_mask,
            pad_token_id=tokenizer.pad_token_id,
            temperature=temperature,
        )

        # token IDs를 text로 변환
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        poem += generated_text
        remaining_words -= len(generated_text.split())

        # 생성된 text의 마지막 부분으로 프롬프트를 업데이트
        prompt = generated_text.split()[-max_seq_len:]

    return poem

import re

def post_process_poem(poem):
    # 여분의 스페이스(공백) 제거
    poem = re.sub(r'\s+', ' ', poem).strip()

    # 각 sentence의 첫 글자를 대문자로 변경
    sentences = re.split(r'(?<=[\.\?!])\s', poem)
    formatted_sentences = [sentence.capitalize() for sentence in sentences]
    formatted_poem = ' '.join(formatted_sentences)

    # 가독성을 위해 줄 변경(line breaks) 조치
    line_breaks = re.compile(r'(?<=[,;:?!])\s')
    formatted_poem = line_breaks.sub('\n', formatted_poem)

    return formatted_poem


In [ ]:
# 예제(Example usage)

### 다음 줄은 역자가 삽입한 코드임
from transformers import GPT2Tokenizer

model_path = '/content/drive/MyDrive/Book6/Ch6/model_checkpoint_epoch_10'
tokenizer_path = 'gpt2'
prompt = "love"
max_words = 50
temperature = 0.9  # 이 수치는 randomness 확장 혹은 축소를 위해 조정 가능
generated_poem = generate_poem(prompt, model_path, tokenizer_path, max_words=max_words, temperature=temperature)
formatted_poem = post_process_poem(generated_poem)
print(formatted_poem)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Love ill and be of:
weeds that in grow in soil will this be for. Rom:well you along ard:
rootloveillandbethatingrowinwillthisbeyoualongrootofarthomei orthomem by of,
of!othoth grow oldbygrowold is nurse with delivery i bury--,
with suck tongue patience patience learn in than pastunniswithiwithlearninthanby oftiswhich titles havell:
tyranny tyranny reign one tyranny
